# Internet of Things (2IMN25)
### Machine Learning Assignment

### Introduction

*Add assignment description here*

In [1]:
""" Import all libraries here """
import numpy as np
import os
import math
from datetime import datetime
import pandas as pd
from dateutil import parser


## Reading the dataset

In the code box below, please read in the data-sets into suitable data structures. The key task here would be to contruct the feature vectors which would be fed to the regression algorithm.

The datasets for the weather, rain and energy consumption have been provided in the zip file, extract this zip file into the same folder as the python notebook. 

*Hint : Be sure to check if there are any missing fields in the provided data*

In [2]:
data_path = os.path.join(os.getcwd(),'data')

""" Read in the weather"""
weather_array = pd.read_csv(os.path.join(data_path,'Weather-Eindhoven.csv'),
                         delimiter = ',',
                         header=0,
                         parse_dates = [0],
                         usecols = [0,2,4,6,7,8]) #Drop "chill", too many missing values
weather_array  = weather_array.as_matrix()

""" Read in the rain data"""
rain_array = pd.read_csv(os.path.join(data_path,'Rain-Best.csv'),
                         delimiter = ',',
                         header=0,
                         parse_dates = [0])

rain_array = rain_array.as_matrix()

""" Synchronize the 2 data-sets by removing the extra data from the rainfall dataset"""
remove_rows = []
for element,idx in zip(rain_array[:,0],range(len(rain_array[:,0]))):
    if element.minute%10 != 0:
        remove_rows.append(idx)

synced_rain_array = np.delete(rain_array,remove_rows,axis = 0)

print(synced_rain_array.shape)
print(weather_array[:,0].shape)

""" Merge the weather and rain data when time-stamps are equal """
merged_array = []
for rainElem in synced_rain_array:
    try :
        merged_row = []
        valid_row = list(weather_array[:,0]).index(rainElem[0])
        for weatherFeature in weather_array[valid_row]:
            merged_row.append(weatherFeature)
        merged_row.append(rainElem[1])
        merged_array.append(np.asarray(merged_row))
    except ValueError:
        print('Did not find matching timestamp in the weather data, continuing')
        continue

merged_array = np.asarray(merged_array)

(43844, 2)
(43872,)
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matchin

In [3]:
""" Clean up the data by removing rows with missing data points """
missing_data_rows = []
for row,idx in zip(merged_array,range(merged_array.shape[0])):
    if row[0].month > 10:
        missing_data_rows.append(idx)
        continue # No energy data for the month of November so delete those too.
    for elem in row[1:]: #Timestamp cannot be "checked"
        if math.isnan(elem):
            missing_data_rows.append(idx)
            break # To prevent the same row from being added multiple times to the list, break when the first "nan" is found
clean_data_array = np.delete(merged_array,missing_data_rows,axis=0)
print(clean_data_array.shape)

(43501, 7)


In [4]:
def create_dataframe(energyDir):
    """ Returns a dataframe that is formed by concatenating all the energy data files """
    fileList = []
    energyDir = os.path.join(os.getcwd(),energyDir)
    dirList = [os.path.join(energyDir, o) for o in os.listdir(energyDir) 
                    if os.path.isdir(os.path.join(energyDir,o))]
    
    sortedDirList = sorted(dirList)
    for direc in sortedDirList:
        dirFiles = [os.path.join(direc, f) for f in os.listdir(direc) 
                    if os.path.isfile(os.path.join(direc,f))]
        for f in dirFiles:
            fileList.append(f)
        
    fileList = sorted(fileList)
    frame = pd.DataFrame()
    frameList = []
    for f in fileList:
        try:
            df = pd.read_csv(f,index_col=None, header=None,parse_dates=[[0,1]],skipfooter=2)
            frameList.append(df)
        except:
            continue
    frame = pd.concat(frameList)
    sparseFrame = frame.drop([2,3,4,5,6,7,8,9,12,13],axis=1)
    sparseFrame.columns = ['TimeStamp','Energy Consumed','Energy Produced']
    return sparseFrame
    
energyFrame = create_dataframe('data/2017')
    


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


In [19]:
#energyFrame['TimeStamp'] = pd.to_datetime(energyFrame['TimeStamp'])
#subFrame = energyFrame[(energyFrame['TimeStamp']>clean_data_array[0][0]) & (energyFrame['TimeStamp']<clean_data_array[1][0])]
def energy_lookup(data_array,energyFrame,idx):
    """ Given the index, calculated the net-energy for that interval """
    try:
        subFrame = energyFrame[(energyFrame['TimeStamp']>data_array[idx-1][0]) & (energyFrame['TimeStamp']<data_array[idx][0])]
        if subFrame.shape[0] == 0:
            return None
        energyList = []
        for cons,prod in zip(subFrame['Energy Consumed'],subFrame['Energy Produced']):
            energyList.append(cons-prod)
        return np.sum(np.asarray(energyList))
    except:
        return None

def create_labels(data_array,energyFrame):
    missing_labels = []
    labels = []
    labels.append(0)
    for row,idx in zip(data_array,range(data_array.shape[0])):
        if idx == 0:
            continue
        netEnergy = energy_lookup(data_array,energyFrame,idx)
        if netEnergy is None:
            print('Missing label for row {}'.format(idx))
            missing_labels.append(idx)
        else:
            print('{} {}'.format(row[0],netEnergy))
            labels.append(netEnergy)
    return labels,missing_labels

labels,missing_labels = create_labels(clean_data_array,energyFrame)
        
            

2017-01-01 00:10:00 29060
2017-01-01 00:20:00 22260
2017-01-01 00:30:00 77630
2017-01-01 00:40:00 47870
2017-01-01 00:50:00 39400
2017-01-01 01:00:00 33420
2017-01-01 01:10:00 37830
2017-01-01 01:20:00 32670
2017-01-01 01:30:00 28250
2017-01-01 01:40:00 29980
2017-01-01 01:50:00 21270
2017-01-01 02:00:00 19600
2017-01-01 02:10:00 23560
2017-01-01 02:20:00 29330
2017-01-01 02:30:00 35320
2017-01-01 02:40:00 21480
2017-01-01 02:50:00 20330
2017-01-01 03:00:00 23330
2017-01-01 03:10:00 22760
2017-01-01 03:20:00 19360
2017-01-01 03:30:00 18870
2017-01-01 03:40:00 18740
2017-01-01 03:50:00 15930
2017-01-01 04:00:00 12600
2017-01-01 04:10:00 12700
2017-01-01 04:20:00 17000
2017-01-01 04:30:00 12600
2017-01-01 04:40:00 13820
2017-01-01 04:50:00 21200
2017-01-01 05:00:00 16790
2017-01-01 05:10:00 16310
2017-01-01 05:20:00 47020
2017-01-01 05:30:00 16890
2017-01-01 05:40:00 11400
2017-01-01 05:50:00 15670
2017-01-01 06:00:00 12030
2017-01-01 06:10:00 11400
2017-01-01 06:20:00 14370
2017-01-01 0

Missing label for row 318
Missing label for row 319
Missing label for row 320
Missing label for row 321
Missing label for row 322
Missing label for row 323
Missing label for row 324
Missing label for row 325
Missing label for row 326
Missing label for row 327
Missing label for row 328
Missing label for row 329
Missing label for row 330
Missing label for row 331
Missing label for row 332
Missing label for row 333
Missing label for row 334
Missing label for row 335
Missing label for row 336
Missing label for row 337
Missing label for row 338
Missing label for row 339
Missing label for row 340
Missing label for row 341
Missing label for row 342
Missing label for row 343
2017-01-03 09:20:00 11130
2017-01-03 09:30:00 23630
2017-01-03 09:40:00 15210
2017-01-03 09:50:00 3340
2017-01-03 10:00:00 14260
2017-01-03 10:10:00 -19460
2017-01-03 10:20:00 -31590
2017-01-03 10:30:00 -50300
2017-01-03 10:40:00 -61460
2017-01-03 10:50:00 -31550
2017-01-03 11:00:00 32759
2017-01-03 11:10:00 94590
2017-01-

2017-01-05 12:20:00 -89140
2017-01-05 12:30:00 -83970
2017-01-05 12:40:00 -80790
2017-01-05 12:50:00 -83120
2017-01-05 13:00:00 -66990
2017-01-05 13:10:00 -59680
2017-01-05 13:20:00 -45950
2017-01-05 13:30:00 -28590
2017-01-05 13:40:00 -38390
2017-01-05 13:50:00 -6910
2017-01-05 14:00:00 16320
2017-01-05 14:10:00 -3040
2017-01-05 14:20:00 -50660
2017-01-05 14:30:00 -48990
2017-01-05 14:40:00 -26420
2017-01-05 14:50:00 -14280
2017-01-05 15:00:00 -1220
2017-01-05 15:10:00 4640
2017-01-05 15:20:00 13670
2017-01-05 15:30:00 21220
2017-01-05 15:40:00 19020
2017-01-05 15:50:00 20610
2017-01-05 16:00:00 21410
2017-01-05 16:10:00 60480
2017-01-05 16:20:00 26900
2017-01-05 16:30:00 19810
2017-01-05 16:40:00 17580
2017-01-05 16:50:00 46310
2017-01-05 17:00:00 22130
2017-01-05 17:10:00 18170
2017-01-05 17:20:00 21800
2017-01-05 17:30:00 18600
2017-01-05 17:40:00 26890
2017-01-05 17:50:00 24610
2017-01-05 18:00:00 20470
2017-01-05 18:10:00 24390
2017-01-05 18:20:00 22840
2017-01-05 18:30:00 21310


2017-01-07 16:40:00 27060
2017-01-07 16:50:00 31520
2017-01-07 17:00:00 32800
2017-01-07 17:10:00 33010
2017-01-07 17:20:00 36800
2017-01-07 17:30:00 33130
2017-01-07 17:40:00 35880
2017-01-07 17:50:00 32750
2017-01-07 18:00:00 32750
2017-01-07 18:10:00 36990
2017-01-07 18:20:00 52880
2017-01-07 18:30:00 36660
2017-01-07 18:40:00 34740
2017-01-07 18:50:00 34240
2017-01-07 19:00:00 130740
2017-01-07 19:10:00 44260
2017-01-07 19:20:00 42820
2017-01-07 19:30:00 68280
2017-01-07 19:40:00 76890
2017-01-07 19:50:00 81440
2017-01-07 20:00:00 32170
2017-01-07 20:10:00 74180
2017-01-07 20:20:00 105530
2017-01-07 20:30:00 139600
2017-01-07 20:40:00 136730
2017-01-07 20:50:00 100820
2017-01-07 21:00:00 38830
2017-01-07 21:10:00 52560
2017-01-07 21:20:00 68180
2017-01-07 21:30:00 47320
2017-01-07 21:40:00 40260
2017-01-07 21:50:00 166920
2017-01-07 22:00:00 135740
2017-01-07 22:10:00 34000
2017-01-07 22:20:00 35920
2017-01-07 22:30:00 35750
2017-01-07 22:40:00 32190
2017-01-07 22:50:00 32630
2017-

2017-01-09 22:40:00 48640
2017-01-09 22:50:00 47810
2017-01-09 23:00:00 22350
2017-01-09 23:10:00 14190
2017-01-09 23:20:00 13800
2017-01-09 23:30:00 18880
2017-01-09 23:40:00 14250
2017-01-09 23:50:00 15700
2017-01-10 00:00:00 18450
2017-01-10 00:10:00 15440
2017-01-10 00:20:00 16070
2017-01-10 00:30:00 64200
2017-01-10 00:40:00 15860
2017-01-10 00:50:00 8000
2017-01-10 01:00:00 10430
2017-01-10 01:10:00 10480
2017-01-10 01:20:00 12130
2017-01-10 01:30:00 14640
2017-01-10 01:40:00 16700
2017-01-10 01:50:00 12610
2017-01-10 02:00:00 11530
2017-01-10 02:10:00 12310
2017-01-10 02:20:00 8360
2017-01-10 02:30:00 7690
2017-01-10 02:40:00 13390
2017-01-10 02:50:00 36820
2017-01-10 03:00:00 13250
2017-01-10 03:10:00 8100
2017-01-10 03:20:00 11750
2017-01-10 03:30:00 7250
2017-01-10 03:40:00 7320
2017-01-10 03:50:00 12950
2017-01-10 04:00:00 7400
2017-01-10 04:10:00 7440
2017-01-10 04:20:00 11250
2017-01-10 04:30:00 9080
2017-01-10 04:40:00 7450
2017-01-10 04:50:00 9630
2017-01-10 05:00:00 129

...

2017-10-22 19:20:00 60550
2017-10-22 19:30:00 74810
2017-10-22 19:40:00 89350
2017-10-22 19:50:00 116660
2017-10-22 20:00:00 118560
2017-10-22 20:10:00 98460
2017-10-22 20:20:00 45390
2017-10-22 20:30:00 34660
2017-10-22 20:40:00 40410
2017-10-22 20:50:00 40920
2017-10-22 21:00:00 69840
2017-10-22 21:10:00 172310
2017-10-22 21:20:00 105270
2017-10-22 21:30:00 35650
2017-10-22 21:40:00 38740
2017-10-22 21:50:00 39080
2017-10-22 22:00:00 37060
2017-10-22 22:10:00 33130
2017-10-22 22:20:00 29980
2017-10-22 22:30:00 30030
2017-10-22 22:40:00 30680
2017-10-22 22:50:00 25740
2017-10-22 23:00:00 31740
2017-10-22 23:10:00 36180
2017-10-22 23:20:00 34120
2017-10-22 23:30:00 48280
2017-10-22 23:40:00 27130
2017-10-22 23:50:00 23500
2017-10-23 00:00:00 28090
2017-10-23 00:10:00 16180
2017-10-23 00:20:00 14560
2017-10-23 00:30:00 17990
2017-10-23 00:40:00 21380
2017-10-23 00:50:00 12400
2017-10-23 01:00:00 7810
2017-10-23 01:10:00 10660
2017-10-23 01:20:00 9840
2017-10-23 01:30:00 7800
2017-10-23 

2017-10-25 04:00:00 11260
2017-10-25 04:10:00 7320
2017-10-25 04:20:00 7200
2017-10-25 04:30:00 10120
2017-10-25 04:40:00 8880
2017-10-25 04:50:00 7200
2017-10-25 05:20:00 40270
2017-10-25 05:50:00 34460
2017-10-25 06:00:00 12560
2017-10-25 06:10:00 22230
2017-10-25 06:20:00 21620
2017-10-25 06:30:00 8640
2017-10-25 06:50:00 28560
2017-10-25 07:00:00 9030
2017-10-25 07:10:00 7200
2017-10-25 07:20:00 8970
2017-10-25 07:30:00 10440
2017-10-25 07:40:00 33160
2017-10-25 07:50:00 68690
2017-10-25 08:00:00 13490
2017-10-25 08:10:00 17570
2017-10-25 08:20:00 17730
2017-10-25 08:30:00 23470
2017-10-25 08:40:00 12630
2017-10-25 08:50:00 28070
2017-10-25 09:00:00 10650
2017-10-25 09:10:00 14620
2017-10-25 09:20:00 12610
2017-10-25 09:30:00 10620
2017-10-25 09:40:00 13660
2017-10-25 09:50:00 14790
2017-10-25 10:00:00 9180
2017-10-25 10:10:00 -5550
2017-10-25 10:20:00 -12130
2017-10-25 10:30:00 -9930
2017-10-25 10:50:00 -36590
2017-10-25 11:00:00 -7100
2017-10-25 11:10:00 1500
2017-10-25 11:20:00 

2017-10-27 12:00:00 -95060
2017-10-27 12:10:00 -66160
2017-10-27 12:20:00 -77280
2017-10-27 12:30:00 -60060
2017-10-27 12:40:00 -35540
2017-10-27 12:50:00 -63210
2017-10-27 13:00:00 -52019
2017-10-27 13:10:00 -25510
2017-10-27 13:20:00 -12830
2017-10-27 13:30:00 -30320
2017-10-27 13:40:00 -65360
2017-10-27 13:50:00 -68500
2017-10-27 14:00:00 -90290
2017-10-27 14:10:00 -91250
2017-10-27 14:20:00 -72860
2017-10-27 14:30:00 -37590
2017-10-27 14:40:00 -80930
2017-10-27 14:50:00 -56840
2017-10-27 15:00:00 -6360
2017-10-27 15:10:00 -8630
2017-10-27 15:20:00 -74090
2017-10-27 15:30:00 -66390
2017-10-27 15:40:00 -59140
2017-10-27 15:50:00 -55720
2017-10-27 16:00:00 -37430
2017-10-27 16:10:00 -35850
2017-10-27 16:20:00 -31110
2017-10-27 16:30:00 2580
2017-10-27 16:40:00 16820
2017-10-27 16:50:00 34020
2017-10-27 17:00:00 13650
2017-10-27 17:10:00 13590
2017-10-27 17:20:00 6250
2017-10-27 17:30:00 6740
2017-10-27 17:40:00 7330
2017-10-27 17:50:00 9470
2017-10-27 18:00:00 5640
2017-10-27 18:10:00

Missing label for row 43153
Missing label for row 43154
Missing label for row 43155
Missing label for row 43156
Missing label for row 43157
Missing label for row 43158
Missing label for row 43159
Missing label for row 43160
Missing label for row 43161
Missing label for row 43162
Missing label for row 43163
Missing label for row 43164
Missing label for row 43165
Missing label for row 43166
Missing label for row 43167
Missing label for row 43168
Missing label for row 43169
Missing label for row 43170
Missing label for row 43171
Missing label for row 43172
Missing label for row 43173
Missing label for row 43174
Missing label for row 43175
Missing label for row 43176
Missing label for row 43177
Missing label for row 43178
Missing label for row 43179
Missing label for row 43180
Missing label for row 43181
Missing label for row 43182
Missing label for row 43183
Missing label for row 43184
Missing label for row 43185
Missing label for row 43186
Missing label for row 43187
Missing label for ro

Missing label for row 43456
Missing label for row 43457
Missing label for row 43458
Missing label for row 43459
Missing label for row 43460
Missing label for row 43461
Missing label for row 43462
Missing label for row 43463
Missing label for row 43464
Missing label for row 43465
Missing label for row 43466
Missing label for row 43467
Missing label for row 43468
Missing label for row 43469
Missing label for row 43470
Missing label for row 43471
Missing label for row 43472
Missing label for row 43473
Missing label for row 43474
Missing label for row 43475
Missing label for row 43476
Missing label for row 43477
Missing label for row 43478
Missing label for row 43479
Missing label for row 43480
Missing label for row 43481
Missing label for row 43482
Missing label for row 43483
Missing label for row 43484
Missing label for row 43485
Missing label for row 43486
Missing label for row 43487
Missing label for row 43488
Missing label for row 43489
Missing label for row 43490
Missing label for ro

In [21]:
featureArray = np.delete(clean_data_array,missing_labels,axis=0)

In [22]:
print(len(labels))
print(featureArray.shape)

40209
(40209, 7)
